In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -U --quiet accelerate
! pip install -U --quiet transformers
! pip install -U --quiet evaluate
! pip install -U --quiet  datasets
! pip install --quiet langchain
! pip install --quiet langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00


In [3]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, DataCollatorWithPadding, Trainer, TrainingArguments,AutoTokenizer
import evaluate
import torch

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForQuestionAnswering.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load base model

In [4]:
import nltk
nltk.download('punkt')

c = "Welcome to NovaGaea, a celestial marvel that seamlessly combines the terrestrial charm of Earth with the grandiosity of Jupiter. This fictitious planet, situated in the cosmic tableau, presents a captivating fusion of planetary properties that sparks the imagination.With a diameter matching Earth's at approximately 12,742 kilometers, NovaGaea offers a familiar sense of solidity underfoot. Its gravitational pull, akin to Earth's at 9.81 m/s², provides a stable environment conducive to diverse landscapes. Sprawling forests, expansive oceans, and towering mountain ranges reminiscent of Earth adorn the surface. As the sun sets on NovaGaea, Jupiter's influence becomes apparent. Nights are illuminated by auroras, akin to those found in Earth's polar regions but on a grander scale. Massive storms, reminiscent of Jupiter's iconic tempests, create celestial displays with winds reaching speeds of 600 kilometers per hour, painting the skies with otherworldly brilliance. NovaGaea's atmosphere, composed of 78% nitrogen, 21% oxygen, and trace amounts of other gases, mirrors Earth's, fostering a rich biosphere. However, the presence of swirling gas clouds high above adds an extraterrestrial touch, creating a delicate balance between the familiar and the extraordinary. Numerical details extend below the surface. NovaGaea's metallic hydrogen core, mirroring Jupiter's composition, contributes to a gravitational force that is a harmonious blend of Earth's stability and Jupiter's grandeur. With a mass approaching 1.898 x 10^27 kilograms, NovaGaea stands as a celestial hybrid, inviting explorers to unravel the secrets hidden within its cosmic embrace."

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from datasets import load_dataset
data_files = {"train": "/content/drive/MyDrive/colab/qa.csv", "test": "/content/drive/MyDrive/colab/qa_test.csv"}
dataset = load_dataset("csv", data_files=data_files)
print(dataset)
def preprocess(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        [c]*len(questions),
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        start_char = int(examples["answerStart"][i])
        end_char = min(int(examples["answerEnd"][i]), len(c)-1)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)
print(tokenized_dataset)
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Index', 'question', 'answerStart', 'answerEnd'],
        num_rows: 101
    })
    test: Dataset({
        features: ['Index', 'question', 'answerStart', 'answerEnd'],
        num_rows: 20
    })
})


Map:   0%|          | 0/101 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 101
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 20
    })
})


In [6]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 101
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 20
    })
})

In [7]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=30,              # total number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy="no",
    logging_steps=20,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
20,5.471800
40,4.605500
60,4.068100
80,3.869600
100,3.628900
120,3.491500
140,3.332200
160,3.247900
180,3.040000
200,2.880100


TrainOutput(global_step=390, training_loss=3.0440033839299128, metrics={'train_runtime': 114.5169, 'train_samples_per_second': 26.459, 'train_steps_per_second': 3.406, 'total_flos': 296909169822720.0, 'train_loss': 3.0440033839299128, 'epoch': 30.0})

In [10]:
q = "what is the diameter of NovaGaea?"
t = tokenizer(q,c,return_tensors="pt")

with torch.no_grad():
  outputs = trainer.model(**t.to("cuda"))

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = t.input_ids[0, answer_start_index : answer_end_index + 1]
print (predict_answer_tokens)
tokenizer.decode(predict_answer_tokens)


tensor([ 2007,  1037,  6705,  9844,  3011,  1005,  1055,  2012,  3155,  2260,
         1010,  6356,  2475,  7338,  1010,  6846,  3654,  5243,  4107,  1037,
         5220,  3168,  1997,  5024,  3012,  2104, 13064,  1012,  2049, 19790,
         4139,  1010, 17793,  2000,  3011,  1005,  1055,  2012,  1023,  1012,
         6282,  1049,  1013,  1055, 10701,  1010,  3640,  1037,  6540,  4044,
         9530,  8566,  6895,  3726,  2000,  7578, 12793,  1012, 24199,  6138,
         1010, 25145, 17401,  1010,  1998, 20314,  3137,  8483, 14563,  1997,
         3011,  4748,  9691,  1996,  3302,  1012,  2004,  1996,  3103,  4520,
         2006,  6846,  3654,  5243,  1010, 13035,  1005,  1055],
       device='cuda:0')


"with a diameter matching earth's at approximately 12, 742 kilometers, novagaea offers a familiar sense of solidity underfoot. its gravitational pull, akin to earth's at 9. 81 m / s², provides a stable environment conducive to diverse landscapes. sprawling forests, expansive oceans, and towering mountain ranges reminiscent of earth adorn the surface. as the sun sets on novagaea, jupiter's"

In [9]:
q = "How does NovaGaea compare to Earth in diameter?"
t = tokenizer(q, c, return_tensors="pt")

with torch.no_grad():
  outputs = trainer.model(**t.to("cuda"))

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = t.input_ids[0, answer_start_index : answer_end_index + 1]
print (predict_answer_tokens)
tokenizer.decode(predict_answer_tokens)


tensor([ 2007,  1037,  6705,  9844,  3011,  1005,  1055,  2012,  3155,  2260,
         1010,  6356,  2475,  7338,  1010,  6846,  3654,  5243,  4107,  1037,
         5220,  3168,  1997,  5024,  3012,  2104, 13064,  1012,  2049, 19790,
         4139,  1010, 17793,  2000,  3011,  1005,  1055,  2012,  1023,  1012,
         6282,  1049,  1013,  1055, 10701,  1010,  3640,  1037,  6540,  4044,
         9530,  8566,  6895,  3726,  2000,  7578, 12793,  1012, 24199,  6138,
         1010, 25145, 17401,  1010,  1998, 20314,  3137,  8483, 14563,  1997,
         3011,  4748,  9691,  1996,  3302,  1012,  2004,  1996,  3103,  4520,
         2006,  6846,  3654,  5243,  1010, 13035,  1005,  1055,  3747,  4150,
         6835,  1012,  6385], device='cuda:0')


"with a diameter matching earth's at approximately 12, 742 kilometers, novagaea offers a familiar sense of solidity underfoot. its gravitational pull, akin to earth's at 9. 81 m / s², provides a stable environment conducive to diverse landscapes. sprawling forests, expansive oceans, and towering mountain ranges reminiscent of earth adorn the surface. as the sun sets on novagaea, jupiter's influence becomes apparent. nights"

In [11]:
q = "what is the predominant gas in novagaea?"
t = tokenizer(q, c, return_tensors="pt")

with torch.no_grad():
  outputs = trainer.model(**t.to("cuda"))

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = t.input_ids[0, answer_start_index : answer_end_index + 1]
print (predict_answer_tokens)
tokenizer.decode(predict_answer_tokens)


tensor([ 6275,  1003, 14114,  1010,  2538,  1003,  7722], device='cuda:0')


'78 % nitrogen, 21 % oxygen'

In [21]:
q = "what is the % of oxygen in novagaea"
t = tokenizer(q, c, return_tensors="pt")

with torch.no_grad():
  outputs = trainer.model(**t.to("cuda"))

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = t.input_ids[0, answer_start_index : answer_end_index + 1]
print (predict_answer_tokens)
tokenizer.decode(predict_answer_tokens)


tensor([2538, 1003, 7722], device='cuda:0')


'21 % oxygen'

In [23]:
q = "what is the landscape of novagaea ?"
t = tokenizer(q, c, return_tensors="pt")

with torch.no_grad():
  outputs = trainer.model(**t.to("cuda"))

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = t.input_ids[0, answer_start_index : answer_end_index + 1]
print (predict_answer_tokens)
tokenizer.decode(predict_answer_tokens)


tensor([ 1010,  4350,  1999,  1996, 14448,  2795,  4887,  1010,  7534,  1037,
        14408, 17441, 10077,  1997, 17700,  5144,  2008, 12300,  1996,  9647,
         1012,  2007,  1037,  6705,  9844,  3011,  1005,  1055,  2012,  3155,
         2260,  1010,  6356,  2475,  7338,  1010,  6846,  3654,  5243,  4107,
         1037,  5220,  3168,  1997,  5024,  3012,  2104, 13064,  1012,  2049,
        19790,  4139,  1010, 17793,  2000,  3011,  1005,  1055,  2012,  1023,
         1012,  6282,  1049,  1013,  1055, 10701,  1010,  3640,  1037,  6540,
         4044,  9530,  8566,  6895,  3726,  2000,  7578, 12793,  1012, 24199,
         6138,  1010, 25145, 17401,  1010,  1998, 20314,  3137,  8483, 14563,
         1997,  3011,  4748,  9691,  1996,  3302,  1012,  2004,  1996,  3103,
         4520,  2006,  6846,  3654,  5243,  1010, 13035,  1005,  1055],
       device='cuda:0')


", situated in the cosmic tableau, presents a captivating fusion of planetary properties that sparks the imagination. with a diameter matching earth's at approximately 12, 742 kilometers, novagaea offers a familiar sense of solidity underfoot. its gravitational pull, akin to earth's at 9. 81 m / s², provides a stable environment conducive to diverse landscapes. sprawling forests, expansive oceans, and towering mountain ranges reminiscent of earth adorn the surface. as the sun sets on novagaea, jupiter's"